In [9]:
import os
import time
import traceback
import snowflake.connector
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from pinecone import Pinecone
from dotenv import load_dotenv


def check_index_stats():
    stats = index.describe_index_stats()
    print(f"Current index stats: {stats}")
    return stats['total_vector_count']

def upsert_embeddings(vectors, batch_number):
    try:
        # Check for duplicate IDs
        ids = [v[0] for v in vectors]
        if len(ids) != len(set(ids)):
            print(f"Batch {batch_number} contains duplicate IDs!")
        
        response = index.upsert(vectors=vectors)
        print(f"Batch {batch_number} - Upsert response: {response}")
        time.sleep(1)  # Add a small delay after each upsert
        current_count = check_index_stats()
        print(f"Batch {batch_number} - Current vector count: {current_count}")
        return response, current_count
    except Exception as e:
        print(f"Error upserting embeddings for batch {batch_number}:")
        print(traceback.format_exc())
        return None, None

def main():
    print("Initial index stats:")
    initial_count = check_index_stats()

    offset = 0
    total_records = min(VECTOR_LIMIT, 1000000)
    batch_number = 0
    total_processed = 0
    all_ids = set()

    while offset < total_records:
        batch_number += 1
        print(f"\nProcessing batch {batch_number}")
        
        df = read_data_from_snowflake(offset, BATCH_SIZE)
        
        if df.empty:
            print("No more data to process.")
            break

        records_in_batch = len(df)
        print(f"Records in this batch: {records_in_batch}")

        # Check for duplicate IDs across batches
        batch_ids = set(df['RECIPEID'].astype(str))
        duplicate_ids = batch_ids.intersection(all_ids)
        if duplicate_ids:
            print(f"Batch {batch_number} contains {len(duplicate_ids)} IDs that were seen in previous batches.")
        all_ids.update(batch_ids)

        text_to_embed = [
            ' '.join([str(row[col]) for col in ['NAME', 'DESCRIPTION', 'RECIPECATEGORY', 'KEYWORDS'] if not pd.isnull(row[col])])
            for _, row in df.iterrows()
        ]
        
        try:
            embeddings = generate_embeddings(text_to_embed)
            vectors = [(str(row['RECIPEID']), embeddings[idx].tolist()) for idx, row in df.iterrows()]
            
            upsert_response, current_count = upsert_embeddings(vectors, batch_number)
            if upsert_response:
                total_processed += records_in_batch
                print(f"Total records processed so far: {total_processed}")
                if current_count and current_count < total_processed:
                    print(f"WARNING: Discrepancy detected. Processed: {total_processed}, Stored: {current_count}")
            
        except Exception as e:
            print(f"Error processing batch {batch_number} starting at index {offset}: {e}")
            print(traceback.format_exc())
        
        offset += BATCH_SIZE
        time.sleep(2)  # Add a delay between batches

    print("\nEmbeddings generation and storage process completed.")

    print("Final index stats:")
    final_count = check_index_stats()

    print(f"Initial vector count: {initial_count}")
    print(f"Final vector count: {final_count}")
    print(f"Vectors added according to Pinecone: {final_count - initial_count}")
    print(f"Total records processed according to our script: {total_processed}")

if __name__ == "__main__":
    main()

Initial index stats:
Current index stats: {'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1229}},
 'total_vector_count': 1229}

Processing batch 1


/var/folders/d1/713jb3f916n662_fxtm8kgy40000gn/T/ipykernel_15624/111194393.py:97: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Batch size read from Snowflake: 500 records
Records in this batch: 500
Batch 1 - Upsert response: {'upserted_count': 500}
Current index stats: {'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1229}},
 'total_vector_count': 1229}
Batch 1 - Current vector count: 1229
Total records processed so far: 500

Processing batch 2


/var/folders/d1/713jb3f916n662_fxtm8kgy40000gn/T/ipykernel_15624/111194393.py:97: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Batch size read from Snowflake: 500 records
Records in this batch: 500
Batch 2 - Upsert response: {'upserted_count': 500}
Current index stats: {'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1229}},
 'total_vector_count': 1229}
Batch 2 - Current vector count: 1229
Total records processed so far: 1000

Processing batch 3


...

/var/folders/d1/713jb3f916n662_fxtm8kgy40000gn/T/ipykernel_15624/111194393.py:97: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Batch size read from Snowflake: 500 records
Records in this batch: 500
Batch 164 contains 1 IDs that were seen in previous batches.
Batch 164 contains duplicate IDs!
Batch 164 - Upsert response: {'upserted_count': 500}
Current index stats: {'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1229}},
 'total_vector_count': 1229}
Batch 164 - Current vector count: 1229
Total records processed so far: 77500

Processing batch 165


/var/folders/d1/713jb3f916n662_fxtm8kgy40000gn/T/ipykernel_15624/111194393.py:97: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Batch size read from Snowflake: 500 records
Records in this batch: 500
Batch 165 contains 1 IDs that were seen in previous batches.
Batch 165 contains duplicate IDs!
Batch 165 - Upsert response: {'upserted_count': 500}
Current index stats: {'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1229}},
 'total_vector_count': 1229}
Batch 165 - Current vector count: 1229
Total records processed so far: 78000

Processing batch 166


OperationalError: 250001: Could not connect to Snowflake backend after 1 attempt(s).Aborting